# Snake Eyes Dungeon - save file decoding 

This Jupyter notebook is an exploration on understanding and decoding the save file from the *Snake Eyes Dungeon* game.

There are some [instructions on how to modify the save file](https://steamcommunity.com/sharedfiles/filedetails/?id=2844497474), but those are manual and incomplete. This notebook tries to decode all the saved items.

## Initial setup

In [1]:
import configparser
import os.path
from base64 import b64decode, b64encode
from pathlib import Path
from pprint import pprint

In [2]:
# Convenience functions

def decode(s, times=1):
    '''Repeatedly base64-decode a string multiple times.'''
    ret = s
    for i in range(times):
        ret = b64decode(ret)
    return ret

def encode(s, times=1):
    '''Repeatedly base64-encode a string multiple times.'''
    if isinstance(s, bytes):
        ret = s
    elif isinstance(s, str):
        ret = bytes(s, 'ascii')
    else:
        ret = bytes(str(s), 'ascii')

    for i in range(times):
        ret = b64encode(ret)
    return ret

## Finding the game save file

Different systems have paths. This code here tries several known paths hoping to find one.

See also:

* <https://www.pcgamingwiki.com/wiki/Snake_Eyes_Dungeon#Game_data>
* <https://www.pcgamingwiki.com/wiki/Glossary:Game_data#User_application_data>

In [3]:
savefile_locations = [
    # Linux, native Steam install.
    '~/.local/share/Steam/steamapps/compatdata/740240/pfx/drive_c/users/steamuser/AppData/Local/Snake_Eyes_Dungeon/aacasmlle.bin',
    '$XDG_DATA_HOME/Steam/steamapps/compatdata/740240/pfx/drive_c/users/steamuser/AppData/Local/Snake_Eyes_Dungeon/aacasmlle.bin',
    # Linux, with Steam from flatpak.
    '~/.var/app/com.valvesoftware.Steam/.local/share/Steam/steamapps/compatdata/740240/pfx/drive_c/users/steamuser/AppData/Local/Snake_Eyes_Dungeon/aacasmlle.bin',
    # Windows.
    '%LOCALAPPDATA%/Snake_Eyes_Dungeon/aacasmlle.bin',
]

In [4]:
def find_savefile(locations) -> Path | None:
    for loc in locations:
        p = Path(os.path.expandvars(os.path.expanduser(loc)))
        if p.exists() and p.is_file():
            return p

In [5]:
savefilename = find_savefile(savefile_locations)
print(savefilename)

/home/denilson/.var/app/com.valvesoftware.Steam/.local/share/Steam/steamapps/compatdata/740240/pfx/drive_c/users/steamuser/AppData/Local/Snake_Eyes_Dungeon/aacasmlle.bin


In [6]:
# If we can't find the file, abort.
# There is no point in continuing running the rest of the code without a proper save file.
assert savefilename

## Decoding the game save file

Despite the `.bin` name, the file is actually a plaintext INI-like config file, with CRLF (i.e. `\r\n`) line endings.

In [7]:
# The file is an INI-like file, with DOS-style newlines (CRLF).
parser = configparser.ConfigParser()
parser.read(savefilename)
print(parser.sections())

['a', 'hiscore', 'u', 'p', 't', 'config', 'm', 'd']


In [8]:
pprint(sorted(parser.sections()))

['a', 'config', 'd', 'hiscore', 'm', 'p', 't', 'u']


### a: Artifacts

In [9]:
# 0: 30%+ Coins loot.
# 1: Start with a bomb.
# 2: 50%+ HP Loot.
# 3: -1 damage from Frank and Wolf.
# 4: Always a key on the 12th room.
# 5: 50%+ Damage to Ghosts.
# 6: 50%+ Damage to Mummies.
# 7: 50%+ Damage to Wolves.
# 8: 50%+ Damage to Franks.
# 9: 50%+ Damage to Vamps.
# 10: 10%+ Snake Eyes rolls.
# 11: Start with 40 coins.
# e: The currently enabled artifact.
pprint(dict(parser["a"].items()))

{'0': '"1.000000"',
 '1': '"1.000000"',
 '10': '"1.000000"',
 '11': '"1.000000"',
 '2': '"1.000000"',
 '3': '"1.000000"',
 '4': '"1.000000"',
 '5': '"1.000000"',
 '6': '"1.000000"',
 '7': '"1.000000"',
 '8': '"1.000000"',
 '9': '"1.000000"',
 'e': '"0.000000"'}


### config: Settings

In [10]:
# music, sound, tutofight, tutomap
pprint(dict(parser["config"].items()), width=20)

{'music': '"0.000000"',
 'sound': '"0.000000"',
 'tutofight': '"1.000000"',
 'tutomap': '"1.000000"'}


### d: Dragon? Dungeon?

In [11]:
# e: Reached the game end (i.e. red background at the title screen)
pprint(dict(parser["d"].items()))

{'e': '"1.000000"'}


### hiscore: High score

In [12]:
pprint(dict(parser["hiscore"].items()), width=20)

{'money': '"1435.000000"',
 'rolls': '"97.000000"'}


### m: Missions

In [13]:
pprint({ k: decode(v, 8) if v.startswith('"V') else v for k, v in parser["m"].items()}, width=20)

{'0': b'56',
 '1': b'172',
 '10': b'1',
 '11': b'24',
 '13': b'11',
 '16': b'5',
 '18': b'458',
 '19': b'905',
 '20': b'917',
 '21': b'483',
 '22': b'9',
 '23': b'6',
 '24': b'9',
 '25': b'6',
 '26': b'8',
 '27': b'6',
 '28': b'35',
 '3': b'3',
 '30': b'274',
 '31': b'387',
 '33': b'9',
 '34': b'48',
 '35': b'19',
 '6': b'4',
 '9': b'0',
 'a0': '"48.000000"',
 'a1': '"49.000000"',
 'a2': '"50.000000"',
 'l': '"50.000000"'}


### p: Player

In [14]:
# c: Coins
pprint({ k: decode(v, 7) for k, v in parser['p'].items()})

{'c': b'3627'}


### t: Unknown

In [15]:
pprint(dict(parser["t"].items()))

{'s': '"12.000000"'}


### u: Upgrades

In [16]:
# h: Get extra HP. (0 up to 10)
# a: Get extra Attack. (0 up to 3)
# s: Extra Snake Eyes chances. (0 up to 5)
# l: More coins Loot on battles. (0 up to 10)
# (not available)
# k: Start next quest with the key in your inventory! (0 or 1)
# b: Start next quest with a bomb in your inventory. (0 or 1)
pprint({ k: decode(v, {'a': 11, 'b': 16, 'h': 10, 'k': 15, 'l': 13, 's': 12}[k]) for k, v in parser["u"].items()}, width=20)

{'a': b'3',
 'b': b'0',
 'h': b'10',
 'k': b'0',
 'l': b'10',
 's': b'4'}
